In [2]:
from konlpy.tag import Okt
import pykospacing
import kss
import re
import numpy as np
import pandas as pd

Metal device set to: Apple M2


In [3]:
incheon = pd.read_csv('cluster_data/preprocessed_blog_incheon.csv')
namyangju = pd.read_csv('cluster_data/preprocessed_blog_namyangju.csv')
daegu = pd.read_csv('cluster_data/preprocessed_blog_daegu.csv') 
daejeon = pd.read_csv('cluster_data/preprocessed_blog_daejeon.csv')
yongin = pd.read_csv('cluster_data/preprocessed_blog_yongin.csv')
gwangju = pd.read_csv('cluster_data/preprocessed_blog_gwangju.csv')
ulsan = pd.read_csv('cluster_data/preprocessed_blog_ulsan.csv')
yangyang = pd.read_csv('cluster_data/preprocessed_blog_yangyang.csv')

In [4]:
city = [incheon, 
namyangju,
daegu ,
daejeon ,
yongin ,
gwangju,
ulsan,
yangyang
]

In [5]:
#결측값 삭제
for c in city:
    c.dropna(inplace=True)
    c.reset_index(inplace=True,drop=True)

In [119]:
# 단어 정규화를 통해 정확도 향상
# 띄어쓰기로 인해 분리된 토큰을 하나로 합침.

def text_normalization(city):
    for i in range(len(city)):
        texts = city['preprocessed_content'][i]

        #단어 정규화
        texts = texts.replace('고 군산 군도','고군 산군도').replace('고군산군도','고군산군도')
        texts = texts.replace('군산도','고군산군도')
        texts = texts.replace('바닷가','바다').replace('해수욕장','해안')
        texts = texts.replace('굴','동굴')
        texts = texts.replace('선 유도','선유 도').replace('선유도','선유도')
        texts = texts.replace('무 녀도','무녀 도').replace('무녀도','무녀도')
        texts = texts.replace('비 응도','비응도')
        texts = texts.replace('비 응항','비응 항').replace('비응항','비응항')
        texts = texts.replace('변산 반도','변산반도')
        texts = texts.replace('은파 호수','은파호수')
        texts = texts.replace('스카이 라인', '스카이라인')
        texts = texts.replace('은파 호수 공원','은파호수 공원').replace('은파호수공원','은파호수공원')
        texts = texts.replace('문화 유산','문화유산')
        texts = texts.replace('선셋','일몰')
        texts = texts.replace('공동묘지','묘지')
        texts = texts.replace('반려견','반려동물').replace('반려 견','반려동물')
        texts = texts.replace('애완 견', '반려동물').replace('애완견','반려동물')
        texts = texts.replace('애완동물','반려동물').replace('애완 동물','반려동물')
        texts = texts.replace('애기', '아기')
        texts = texts.replace('초딩', '초등학생')
        texts = texts.replace('칭구', '친구')
        texts = texts.replace('미륵 전', '미륵전')
        texts = texts.replace('채석 강', '채석강')
        texts = texts.replace('해식 동굴', '해식동굴')
        texts = texts.replace('관음 정사', '관음정사')
        texts = texts.replace('관음 봉', '관음봉')
        texts = texts.replace('워터 파크', '워터파크')
        texts = texts.replace('금 산사', '금산 사').replace('금산사','금산사')
        texts = texts.replace('변산 반도','변산반도')
        texts = texts.replace('롯데 월드', '롯데월드')
        texts = texts.replace('농경 문화 박물관', '농경문화 박물관').replace('농경문화박물관','농경문화박물관')
        texts = texts.replace('혜덕 왕사 탑비', '혜덕왕사 탑비').replace('혜덕왕사탑비','혜덕왕사탑비')

In [120]:
for i in city:
    text_normalization(i)

In [6]:
pos_text_list=["","","","","","","",""]

In [7]:
for i in range(len(city)):
    for _, row in city[i].iterrows():
        pos_text_list[i] += row['preprocessed_content']

In [27]:
len(pos_text_list[1])

8357183

# 명사 추출 후 빈도 수 확인

In [8]:
okt = Okt()


systemMemory: 24.00 GB
maxCacheSize: 8.00 GB



In [9]:
myList=[]
myList_filter=[]

In [12]:
for i in range(8):
    myList.append(okt.pos(pos_text_list[i], norm=True, stem=True)) # 모든 형태소 추출
    myList_filter.append([x for x, y in myList[i] if y in ['Noun']]) # 추출된 값 중 명사만 추출

In [13]:
nouns_list=[]

In [14]:
for i in range(8):
    nouns_list.append(' '.join(myList_filter[i]))

In [15]:
word_frequencies_list=[]

In [16]:
from collections import Counter

def count_word_frequencies(corpus):
    # 단어 빈도수를 저장할 Counter 객체 생성
    word_counter = Counter()

    # 말뭉치에서 각 단어 추출하여 빈도수 계산
    words = corpus.split()
    word_counter.update(words)

    # 단어 빈도수를 딕셔너리로 변환
    word_frequency_dict = {word: frequency for word, frequency in word_counter.items()}

    return word_frequency_dict


for i in range(8):
    # 말뭉치 데이터 로드 (예시)
    corpus = nouns_list[i]

    # 단어 빈도수 계산
    word_frequencies = count_word_frequencies(corpus)
    word_frequencies = {key: value for key, value in word_frequencies.items() if value >=50}
    word_frequencies = {key: value for key, value in word_frequencies.items() if len(key) >=2}
    word_frequencies_list.append(word_frequencies)

In [17]:
word_fre_list=[]

In [18]:
for i in range(8):
    word_fre_list.append(sorted(word_frequencies_list[i].items(),reverse=True,key=lambda item: item[1]))
    word_fre_list[i] = dict(word_fre_list[i])

# 피클로 저장

In [21]:
import pickle

with open("nouns_list3.pkl","wb") as f:
    pickle.dump(word_fre_list, f)

In [ ]:
# 단어 정규화와 1차 전처리를 거친 후에도, 각 크롤링 데이터마다 분석에 필요하지 않은 토큰들이 다수 존재했음.
# 빈도 순으로 명사들을 나열하고, 분석에 방해가 될 만한 단어들을 추가적으로 제거했음.

In [48]:
notuse = ['인테리어', '일반', '관광지', '지다', '지도', '중앙', '오늘', '내부', '랜드', '스페이스', '기분', '매표소', '경상북도', 
          '가능', '연어', '리다', '목포시', '제주도', '경상남도', '다람쥐', '경험', '경남', '아빠', '동문', '사실', '사람', '가면', 
          '도로', '현재', '이나', '시작', '나중', '오후', '유리', '월요일', '머리', '대기', '제법', '야외', '근처', '세트', '경북',
          '서로', '보지', '정보', '보호', '친구', '제주', '규모', '후기', '멀리', '가족', '단체', '그냥', '벤치', '출발', '종류', 
          '이름', '제일', '만난', '대해', '관광', '탑승', '건물', '명소', '방향', '점심', '무료', '게임', '목포역진서면', '영업', 
          '작품', '마음', '소녀', '나무', '인증', '재미', '이지', '사이', '강원도', '해상', '그대로', '부산광역시', '안내', '서울특별시',
          '강원', '순천', '주식회사', '김치', '주변', '주소', '포항', '토닉', '골목', '아무', '도시', '숙소', '개인', '이야기', '주문',
          '노량진', '이전', '하니', '주차', '마르다', '물이', '대한', '준비', '아벨', '트릭', '방문', '가성', '부안', '맞이', '참고', 
          '애기', '우암', '지점', '금지', '마을', '전북', '요금', '통영', '대로', '기념', '서서', '오이도', '입구', '요즘', '기도', 
          '음료', '버스', '자리', '라서', '경기', '모험', '실제', '피어', '토요일', '시설', '바닥', '시대', '계속', '사랑', '포리', 
          '거기', '동포', '거리', '생기', '예정', '예전', '전경', '설치', '세종', '가격', '그림', '바람', '도심', '보이', '왕복', 
          '유아', '스테이션', '죽교동', '영광', '대전', '울산', '생활', '추천', '때문', '계시', '찍기', '처럼', '느낌', '어린이', 
          '만큼', '벌써', '숲길', '미리', '발권', '주차장', '용담', '날씨', '부산여행', '먼저', '방법', '관람', '이용', '추석', '어른',
          '아침', '여수', '격포', '길이', '일정', '타고', '처음', '직접', '가장', '다스', '재생', '운영', '정기', '경우', '일상',
          '공사', '목포', '메뉴', '고속도로', '수암', '조각', '장항', '하루', '미호', '모양', '특별자치도', '다시', '문화', '국내',
          '테이블', '최고', '높이', '가득', '구입', '여서', '사장', '식물', '시내', '관련', '영화', '감상', '마이', '포장', '확인', 
          '대구', '티켓', '더욱', '여러', '매일', '입장료', '포항시', '방콕', '사진', '접기', '네이버', '구경', '촬영', '휴무', '남해',
          '설리', '디피', '매장', '구매', '서울', '계획', '사용', '지역', '기억', '서문', '우리나라', '호반', '알다', '이동', '코스',
          '초등학생', '관찰', '최근', '광탄면', '이기', '소리', '만천하', '파다', '모습', '예매', '치즈', '청원구', '방죽', '여행지', 
          '생각', '반기', '면제', '전라남도', '포토', '위해', '위치', '공간', '전남', '국민', '회관', '소사', '청소년', '포스팅', '군산',
          '현대', '인천', '전두환', '요기', '여기', '서귀포시', '주말', '평일', '수도', '공작', '입장권', '여행', '별로', '구성', '순간',
          '시민', '거제', '크리스탈', '설명', '단양', '파주', '햇살', '당시', '장료', '예약', '정도', '오시', '덕동', '엄마', '조류', 
          '가지', '라이딩', '돼지', '도착', '입장', '울진', '지로', '보호자', '워낙', '볼거리', '마련', '보고', '의미', '남아', '의자', 
          '소개', '광주', '정말', '매력', '노래', '문집', '성인', '영상', '아들', '장소', '소요', '지금', '계단', '통해', '정상', 
          '간다', '마당', '아주', '충남', '세계', '금강로', '천원', '오전', '천하', '시흥', '등기', '아우', '개함', '원래', '곳곳', 
          '휴가', '기장', '파주시', '충북', '경기도', '마감', '마지막', '살짝', '소니', '신발', '작년', '만날', '검색', '김제', '관광객',
          '결제', '소나무', '이후', '투어', '카인', '통영시', '아가', '조성', '리뷰', '부담', '캐빈', '중간', '안전', '부산', '역시',
          '제대로', '돗자리', '인지', '구룡포읍', '보기', '대표', '할인', '입다', '마다', '춘천', '출처', '진짜', '암문', '화장실',
          '부분', '금방', '아래', '이제', '다리', '공기', '발견', '모형', '한국', '관람객', '승용차', '커플', '전체', '가게','송도','월미도',
          '왕리','종합','중구','인천광역시','영종도','리해','개구','선녀','분위기','열차','먹기','수산','소래역','']

In [47]:
for c in word_fre_list:
    for key, value in c.items():
        if key not in notuse:
            print(key, value)
    print('--------------------------------')

월미도 9836
송도 8144
소래포구 7346
바다 7019
왕리 6819
해수욕장 6721
파크 6715
맛집 5989
센트럴 5480
어시장 4892
조개 4831
종합 4114
카페 3865
구이 3417
중구 3142
인천광역시 2961
영종도 2789
호텔 2719
새우 2693
공원 2098
보트 2092
칼국수 1932
리해 1766
식당 1658
해변 1630
횟집 1616
해산물 1598
바위 1597
개구 1549
선녀 1488
분위기 1429
튀김 1364
열차 1335
먹기 1296
수산 1286
해물 1285
광역시 1275
대하 1264
저녁 1252
꽃게 1243
식사 1203
음식 1200
연수구 1176
커피 1148
남동구 1144
고기 1108
산책 1102
오션 1094
여름 1076
라면 1074
갈매기 1047
놀이 1041
데이트 1019
풍경 1008
만원 989
물회 982
가리비 966
광어 927
시장 923
양념 920
일몰 907
추가 906
서비스 860
소래역 860
차이나타운 858
소금 858
미도 822
왕산 820
소래 814
기본 813
유서 761
방어 749
소스 747
선택 739
국물 711
나들이 703
근교 696
마리 693
바닷가 685
대게 682
매운탕 672
놀이기구 668
야경 657
하늘 656
디저트 651
완전 645
서해 628
우럭 622
전망대 619
상차림 618
타워 613
직원 609
손님 604
테마 602
판매 599
낙지 597
드라이브 595
청춘 590
반찬 585
모래 584
박물관 578
지하 576
선착장 573
고민 558
전복 550
노을 540
가을 539
생선 527
자주 525
맥주 523
바이킹 522
무한리필 518
남편 516
텐트 500
요리 498
피자 498
리조트 495
아트 494
뭔가 492
메뉴판 491
더위 490
크림 488
겨울 485
킹크랩 485
스끼 483
힐링 483
다가 481
제공 481
해도 480


In [51]:
for j in range(8):
    a = []
    
    for i in word_fre_list[j].keys():
        if i in notuse:
            a.append(i)

    for i in a:
        del word_fre_list[j][i]

In [55]:
from itertools import islice

for i in range(8):
    word_fre_list[i] = dict(islice(word_fre_list[i].items(), 30))

In [56]:
word_fre_list

[{'소래포구': 7346,
  '바다': 7019,
  '해수욕장': 6721,
  '파크': 6715,
  '맛집': 5989,
  '센트럴': 5480,
  '어시장': 4892,
  '조개': 4831,
  '카페': 3865,
  '구이': 3417,
  '호텔': 2719,
  '새우': 2693,
  '공원': 2098,
  '보트': 2092,
  '칼국수': 1932,
  '식당': 1658,
  '해변': 1630,
  '횟집': 1616,
  '해산물': 1598,
  '바위': 1597,
  '튀김': 1364,
  '해물': 1285,
  '광역시': 1275,
  '대하': 1264,
  '저녁': 1252,
  '꽃게': 1243,
  '식사': 1203,
  '음식': 1200,
  '연수구': 1176,
  '커피': 1148},
 {'정원': 8034,
  '물의': 7416,
  '정약용': 5196,
  '남양주': 5078,
  '다산': 3824,
  '유적지': 2970,
  '공원': 2313,
  '남양주시': 2205,
  '양귀비': 2061,
  '조안면': 1965,
  '선생': 1747,
  '산책': 1517,
  '자전거': 1508,
  '북한': 1413,
  '생태': 1363,
  '풍경': 1339,
  '실학': 1206,
  '카페': 1114,
  '나들이': 1095,
  '코스모스': 1092,
  '박물관': 1051,
  '생가': 985,
  '가을': 965,
  '여유당': 937,
  '강변': 935,
  '근교': 895,
  '한강로': 808,
  '수종사': 778,
  '드라이브': 760,
  '맛집': 732},
 {'맛집': 11326,
  '팔공산': 9181,
  '수성': 9037,
  '케이블카': 8800,
  '이월드': 8043,
  '서문시장': 7501,
  '수목원': 7204,
  '엑스코': 6484,
  '카페': 5603,
  '대구

In [57]:
import pickle

with open("hyungbin.pkl","wb") as f:
    pickle.dump(word_fre_list, f)

# 각 도시별 명사 빈도 재 추출

In [253]:
pos_text=""

In [254]:
for _, row in buan.iterrows():
    pos_text += row['preprocessed_content']

# 명사 추출 후 빈도 수 확인

In [255]:
okt = Okt()
myList = okt.pos(pos_text, norm=True, stem=True) # 모든 형태소 추출
myList_filter = [x for x, y in myList if y in ['Noun']] # 추출된 값 중 명사만 추출

In [256]:
nouns = ' '.join(myList_filter)

In [257]:
from collections import Counter

def count_word_frequencies(corpus):
    # 단어 빈도수를 저장할 Counter 객체 생성
    word_counter = Counter()

    # 말뭉치에서 각 단어 추출하여 빈도수 계산
    words = corpus.split()
    words= [x for x in words if x not in notuse]
    word_counter.update(words)

    # 단어 빈도수를 딕셔너리로 변환
    word_frequency_dict = {word: frequency for word, frequency in word_counter.items()}

    return word_frequency_dict

# 말뭉치 데이터 로드 (예시)
corpus = nouns

# 단어 빈도수 계산
word_frequencies = count_word_frequencies(corpus)
word_frequencies = {key: value for key, value in word_frequencies.items() if value >=50}
word_frequencies = {key: value for key, value in word_frequencies.items() if len(key) >=2}

# 결과 출력
#for word, frequency in word_frequencies.items():
 #   print(word, frequency)


In [258]:
sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)

In [259]:
buan_words = sorted_word_frequencies[:70]

In [262]:
yeosu_words = dict(yeosu_words)
gunsan_words = dict(gunsan_words)
buan_words = dict(buan_words)
busan_words = dict(busan_words)
gimje_words = dict(gimje_words)

In [281]:
df1 = pd.DataFrame(yeosu_words,index=['yeosu'])
df2 = pd.DataFrame(gunsan_words,index=['gunsan'])
df3 = pd.DataFrame(busan_words,index=['busan'])
df4 = pd.DataFrame(buan_words,index=['buan'])
df5 = pd.DataFrame(gimje_words,index=['gimje'])

In [283]:
df = pd.concat([df1,df2],join='outer')
df = pd.concat([df,df3],join='outer')
df = pd.concat([df,df4],join='outer')
df = pd.concat([df,df5],join='outer')

In [314]:
df.to_csv('cluster_1.csv', index = False, encoding='utf-8-sig')

In [285]:
df = df.fillna(0)

In [288]:
total_sum = df.sum(axis=1)
keyword_ratio = df.apply(lambda x: round(x/total_sum,2),axis=0)
keyword_ratio

,오동도,바다,공원,이순신,여수시,예술,사진,광장,맛집,전라남도,...,전통,등산로,장소,부량면,방향,구이면,하산,평야,제방,지정
yeosu,0.06,0.05,0.05,0.04,0.04,0.04,0.04,0.03,0.03,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
gunsan,0.00,0.02,0.03,0.00,0.00,0.00,0.04,0.00,0.02,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
busan,0.00,0.05,0.02,0.00,0.00,0.00,0.03,0.00,0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
buan,0.00,0.04,0.01,0.00,0.00,0.00,0.03,0.00,0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
gimje,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.01,0.00,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [289]:
keyword_ratio.sum(axis=1)

yeosu     1.07
gunsan    1.07
busan     1.11
buan      1.12
gimje     1.10
dtype: float64

In [290]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 3, random_state=42)
km.fit(keyword_ratio)

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [303]:
centers = pd.DataFrame(np.round(km.cluster_centers_, 3), columns = km.feature_names_in_)
centers.index.name = "cluster"
centers

,오동도,바다,공원,이순신,여수시,예술,사진,광장,맛집,전라남도,...,전통,등산로,장소,부량면,방향,구이면,하산,평야,제방,지정
cluster,,,,,,,,,,,,,,,,,,,,,
0,0.00,0.04,0.010,0.000,0.000,0.000,0.030,0.00,0.030,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.020,0.000,0.000,0.000,0.020,0.00,0.010,0.00,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,0.02,0.04,0.033,0.013,0.013,0.013,0.037,0.01,0.027,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [312]:
# k = 3
for i in range(3):
    print(pd.DataFrame(centers.iloc[i]).sort_values(by = i, ascending = False)[:8])
    print("-"*30)

         0
변산    0.08
해수욕장  0.07
채석    0.07
바다    0.04
격포항   0.03
사진    0.03
맛집    0.03
전라북도  0.03
------------------------------
        1
축제   0.07
모악산  0.07
지평선  0.05
금산사  0.05
벽골제  0.05
정상   0.03
카페   0.03
체험   0.03
------------------------------
          2
바다    0.040
사진    0.037
공원    0.033
해수욕장  0.033
맛집    0.027
카페    0.023
해운대   0.023
오동도   0.020
------------------------------


In [306]:
keyword_ratio['cluster'] = km.predict(keyword_ratio)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- cluster


In [311]:
keyword_ratio[keyword_ratio['cluster'] == 1].index

Index(['gimje'], dtype='object')